In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from senmodel.model.utils import convert_dense_to_sparse_network, get_model_last_layer
from senmodel.metrics.nonlinearity_metrics import GradientMeanEdgeMetric, PerturbationSensitivityEdgeMetric
from senmodel.metrics.edge_finder import EdgeFinder


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

X = data.drop('income', axis=1)
y = data['income']

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [7]:
class EnhancedFCN(nn.Module):
    def __init__(self, input_size=14, hidden_sizes=None, output_size=2, dropout_rate=0.3):
        super(EnhancedFCN, self).__init__()
        if hidden_sizes is None:
            hidden_sizes = [128, 64, 32]
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_rate)

        self.output = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.output(x)
        return x


In [8]:
def edge_replacement_func_new_layer(model, optim, val_loader, metric):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device)
    vals = ef.calculate_edge_metric_for_dataloader(model)
    print("Edge metrics:", vals, max(vals), sum(vals))
    chosen_edges = ef.choose_edges_threshold(model, 0.05)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if layer.embed_linears:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("Empty metric")
        dummy_param = torch.zeros_like(layer.weight_values)
        optim.add_param_group({'params': dummy_param})

    return {'max': max(vals), 'sum': sum(vals), 'len': len(vals), 'len_choose': len(chosen_edges[0])}


In [9]:
def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=3, threshold=0.1):
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.CrossEntropyLoss()
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(all_targets, all_preds)
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        new_l = {}
        val_losses.append(val_loss)
        if edge_replacement_func and len(val_losses) > window_size:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                new_l = edge_replacement_func(model, optimizer, val_loader, metric)
        #

        # new_l = {}
        # if edge_replacement_func and epoch % 8 == 0 and epoch != 0:
        #     new_l = edge_replacement_func(model, optimizer, val_loader, metric)

        wandb.log({'val_loss': val_loss, 'val_accuracy': val_accuracy, 'train_loss': train_loss} | new_l)



In [10]:
criterion = nn.CrossEntropyLoss()
metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]


In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk. Use `wandb login --relogin` to force relogin


True

In [12]:
dense_model = EnhancedFCN(input_size=X.shape[1])
sparse_model = convert_dense_to_sparse_network(dense_model)
wandb.init(
    project="self-expanding-nets",
    name=f"titanic, complex_model, threshold=0.05, change if loss not changed",
)

train_sparse_recursive(sparse_model, train_loader, val_loader, num_epochs=50,
                       metric=metrics[0],
                       edge_replacement_func=edge_replacement_func_new_layer)

100%|██████████| 51/51 [00:01<00:00, 33.51it/s]


Epoch 1/50 | Train Loss: 0.5603 | Val Loss: 5.9605 | Val Accuracy: 0.7573


100%|██████████| 51/51 [00:01<00:00, 33.76it/s]


Epoch 2/50 | Train Loss: 0.4506 | Val Loss: 5.3475 | Val Accuracy: 0.8224


100%|██████████| 51/51 [00:01<00:00, 34.44it/s]


...

Epoch 48/50 | Train Loss: 0.3848 | Val Loss: 4.4737 | Val Accuracy: 0.8420
Edge metrics: tensor([0.0000e+00, 2.5446e-01, 0.0000e+00, 3.2965e-01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 2.6070e-04, 8.4491e-03, 4.7733e-02, 2.6547e-02, 3.0096e-02,
        4.5651e-02, 1.4171e-02, 2.6828e-02, 1.4626e-02, 1.3929e-02, 4.8887e-01,
        2.5341e-01, 8.2397e-01, 4.7990e-01, 3.9108e-01, 6.8652e-01, 1.7591e-02,
        9.7975e-02, 2.8505e-02, 2.4709e-01, 1.0950e-02, 5.8022e-02, 6.1046e-02,
        3.2367e-02, 1.3728e-03, 8.4321e-02, 1.4310e-01, 5.5838e-02, 1.0029e-01,
        2.1288e-02, 4.0241e-01, 1.0137e-02, 3.0715e-02]) tensor(0.8240) tensor(5.3392)
Chosen edges: tensor([[ 

100%|██████████| 51/51 [00:02<00:00, 19.20it/s]


Epoch 49/50 | Train Loss: 0.3973 | Val Loss: 4.6383 | Val Accuracy: 0.8313
Edge metrics: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0059, 0.0577, 0.0292, 0.0748, 0.0605, 0.0376,
        0.0179, 0.0103, 0.0123, 0.0380, 0.0643, 0.0111, 0.0237, 0.0044, 0.0616,
        0.0076, 0.0552, 0.0000, 0.0000, 0.0000, 0.0702, 0.0000, 0.0904, 0.0533,
        0.4733, 0.0681, 1.0000, 0.0081, 0.0045, 0.3385, 0.1026, 0.0171, 0.0727,
        0.1478]) tensor(1.0000) tensor(3.0187)
Chosen edges: tensor([[  0,   1,   0,   1,   1,   1,   0,   1,   0,   1,   0,   1,   1,   0,
           0,   0],
        [477, 504, 507, 536, 547, 550, 554, 556, 557, 558, 559, 560, 563, 564,
         566, 567]]) 16


100%|██████████| 51/51 [00:02<00:00, 19.56it/s]


Epoch 50/50 | Train Loss: 0.4024 | Val Loss: 4.6143 | Val Accuracy: 0.8428
Edge metrics: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0313, 0.4551, 0.0935, 0.0882, 0.0544, 0.1055,
        0.0735, 0.1308, 0.1384, 0.0090, 0.0394, 0.2189, 0.4691, 0.0000, 0.0000,
        0.0173, 0.0275, 0.1142, 0.3877, 0.0457, 0.2589, 0.0058, 0.0808, 0.0203,
        0.0744, 0.0250, 0.0633, 0.9935, 0.0413, 0.4366, 0.9767, 0.3409, 0.2644,
        0.2947]) tensor(0.9935) tensor(6.3760)
Chosen edges: tensor([[  0,   1,   0,   0,   0,   1,   0,   0,   0,   1,   0,   0,   0,   1,
           0,   0,   1,   1,   1,   0,   0,   0],
        [478, 511, 512, 519, 521, 534, 538, 541, 551, 552, 565, 568, 570, 572,
         574, 576, 577, 579, 580, 581, 582, 583]]) 22
